# Notebook for Work with Optimization and Pool and Executor instances
For possible application in ModelFitting in brian2tools

Notebook goals: 
    - create instances with Scipy and Nevergrad that take two params and return optimal error
    - modify them to use the workers with Executor and Pool 
    - try using NeuronGrouop to adapt them

In [1]:
import numpy as np
import multiprocessing as mp

from scipy.optimize import differential_evolution, rosen
from scipy.optimize._differentialevolution import DifferentialEvolutionSolver

from nevergrad.optimization import optimizerlib
from concurrent import futures
from nevergrad import instrumentation as inst

## Scipy

In [2]:
def test(x):
    arg2 = x[0] - x[1]
    return arg2

In [3]:
bounds = [(-5, 5), (-5, 5)]
result = differential_evolution(test, bounds)
result.x, result.fun

(array([-5.,  5.]), -10.0)

### use workers to map

In [4]:
with mp.Pool(2) as p, DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [6]:
print(solver.x)
print(test(solver.x))

[-5.  5.]
-10.0


### Using The Nevergrad PoolExecutor with Scipy optimize

In [63]:
with futures.ProcessPoolExecutor(max_workers=optim.num_workers) as p, DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [64]:
print(solver.x)
print(test(solver.x))

[-5.  5.]
-10.0


### Nevergrad

In [65]:
# define the variables to be a bounded array
arg1 = inst.var.Array(2).bounded(-5, 5)

instrum = inst.Instrumentation(arg1)
instrum

Instrumentation(Array(shape=(2,), transforms=[ArctanBound(a_max=5, a_min=-5)]))

In [66]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=5000)

In [67]:
optim.optimize(test)

Candidate(args=(array([-4.99990432,  4.99978128]),), kwargs={}, data=[-33268.24273257  14553.30152211])

### Nevergrad with Executor

In [73]:
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000, num_workers=2)

with futures.ProcessPoolExecutor(max_workers=optim.num_workers) as executor:
    recommendation = optim.optimize(test, executor=executor, batch_mode=True)

/home/alteska/anaconda3/envs/brian_opt/lib/python3.6/site-packages/nevergrad/common/tools.py:168: UserWarning: Ignoring since timer was stopped before starting.
  warnings.warn("Ignoring since timer was stopped before starting.")


In [74]:
recommendation

Candidate(args=(array([-4.99999993,  4.99999994]),), kwargs={}, data=[-44026419.57113199  51946212.14350379])

In [75]:
# can not be used the other way around, needs "submit" method

# optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000, num_workers=2)
# with mp.Pool(processes=optim.num_workers) as executor:
#     recommendation = optim.optimize(test, executor=executor, batch_mode=True)

### Two Separate Bounds Problem

In [45]:
arg1 = inst.var.Array(1).bounded(-5,5)
arg2 = inst.var.Array(1).bounded(0,10)
instrum = inst.Instrumentation(arg1, arg2)


In [51]:
def test2(x, y):
    arg2 = x - y
    return arg2

In [62]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=5000)
optim.optimize(test2)

TypeError: test2() missing 1 required positional argument: 'y'